In [1]:
import matplotlib.pyplot as plt
import time

# Tensorflow / Keras
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM

# Data manipulation
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

# file processing
import os

In [5]:
PATH = "test_input"  
file_dir = os.listdir(PATH) 
X_train = np.array([])

for file in file_dir:
    X_train = np.append(X_train, np.load(PATH + "/" + file))

[  0.   0.   0. ...   4.   3. 151.]


In [ ]:
file = ...
train = pd.read_csv(file)

scaler = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
step = 16


model = Sequential()
model.add(LSTM(52, input_shape=(step, 12+12+12+1+1+2+1+1)))
model.add(Dense(52, activation='sigmoid')) 


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=1000, batch_size=10, validation_split=0.05, verbose=1)

In [ ]:
pred_test = model.predict(X)